In [1]:
import yaml
import numpy as np
from matplotlib import pyplot as plt

# --- Custom Utility Module ---
from mineral_utils import Paleodetector

print("Libraries imported successfully.")

Libraries imported successfully.


In [2]:
mineral_name =  "Halite"  # Must match a key in MINERAL_LIBRARY

sample_mass_kg = 0.0001       # Sample mass in kg (e.g., 0.01 for 10g)
total_exposure_kyr = 5600.         # Total age of the sample in millions of years
exposure_window_kyr = 300.  # Duration of CR signal exposure in kyr

# --- Continuous Deposition Model Parameters ---
deposition_rate_m_kyr = np.array([2.5, 30.])
continuous_overburden_density_g_cm3 = 2.16
event_overburden_density_g_cm3 = 1.
event_overburden_depth = 1500.
initial_depth_m =  0.


x_bins = np.linspace(0, 100000, 100)        #Track length bins in nm
x_mids = x_bins[:-1] + np.diff(x_bins) / 2

In [3]:
CONFIG = yaml.safe_load(open('Data/basic_config.yaml', 'r'))
MINERAL_LIBRARY = yaml.safe_load(open('Data/mineral_library.yaml', 'r'))

energies = np.array(CONFIG['geant4_energy_bins_gev'][:90])

In [4]:
scenario_config_simple = {
    'name': 'simple',
    'event_fluxes': {
        0.: ('H3a', 'H3a'),
        300.: ('H3a', 'H3a'),
    }
}

scenario_config_SN20 = {
    'name': 'SN20',
    'event_fluxes': {
        0.: ('SN20pc_2kyr', 'SN20pc_2kyr'),            
        10.: ('SN20pc_12kyr', 'SN20pc_12kyr'),
        20.: ('SN20pc_22kyr', 'SN20pc_22kyr'),
        30.: ('SN20pc_32kyr', 'SN20pc_32kyr'),
        40.: ('SN20pc_42kyr', 'SN20pc_42kyr'),
        50.: ('SN20pc_52kyr', 'SN20pc_52kyr'),
        60.: ('SN20pc_62kyr', 'SN20pc_62kyr'),
        70.: ('H3a', 'H3a'),
        300.: ('H3a', 'H3a'),         
    }
}

scenario_config_SN100 = {
    'name': 'SN100',
    'event_fluxes': {
        0.: ('SN100pc_2kyr', 'SN100pc_2kyr'),            
        10.: ('SN100pc_12kyr', 'SN100pc_12kyr'),
        20.: ('SN100pc_22kyr', 'SN100pc_22kyr'),
        30.: ('SN100pc_32kyr', 'SN100pc_32kyr'),
        40.: ('SN100pc_42kyr', 'SN100pc_42kyr'),
        50.: ('H3a', 'H3a'),
        300.: ('H3a', 'H3a'),   
    }
}

In [5]:
# --- Setup & Verification ---
mineral_config = MINERAL_LIBRARY.get(mineral_name)
if not mineral_config:
    raise ValueError(f"Mineral '{mineral_name}' not found in MINERAL_LIBRARY.")

mineral = Paleodetector(mineral_config)

Initialized Paleodetector: Halite


In [6]:
total_fission_tracks = mineral.integrate_fission_spectrum(x_bins, total_exposure_kyr, sample_mass_kg)

Calculating spontaneous fission background...


In [7]:
total_neutron_tracks = mineral.integrate_background_neutron_spectrum(x_bins, energies, total_exposure_kyr, sample_mass_kg)

/home/cgalelli/Documents/PrimusCode/mineral_utils.py:591: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  total_tracks = [quad(total_tracks_interp, x_bins[i], x_bins[i+1])[0] for i in range(len(x_mids))]


In [8]:
total_tracks_simple = [mineral.integrate_all_particles(
    x_bins, 
    scenario_config=scenario_config_simple, 
    energy_bins_gev=energies, 
    exposure_window_kyr=exposure_window_kyr, 
    sample_mass_kg = sample_mass_kg,
    deposition_rate_m_kyr = deposition,
    overburden_density_g_cm3 = continuous_overburden_density_g_cm3,
    steps=150,
) for deposition in deposition_rate_m_kyr]

Processing species: mu-
Integrating the signal in a 0.001 mm slice of Halite with mass 0.1 g, corresponding to 462.96296296296293 cm2


  0%|          | 0/151 [00:00<?, ?it/s]

    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_2.0kyr_10.8mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_10.0kyr_54.0mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_8.0kyr_43.2mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_6.0kyr_32.4mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_4.0kyr_21.6mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_0.0kyr_0.0mwe.npz


  1%|▏         | 2/151 [01:18<1:21:39, 32.88s/it]

    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_20.0kyr_108.0mwe.npz    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_12.0kyr_64.8mwe.npz

    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_16.0kyr_86.4mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_18.0kyr_97.2mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_14.0kyr_75.6mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_22.0kyr_118.8mwe.npz


...

 95%|█████████▌| 144/151 [34:20<00:57,  8.20s/it]

    - Saved processed data to Data/processed_recoils/Halite/secondary_neutron/simple_290.0kyr_18792.0mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/secondary_neutron/simple_292.0kyr_18921.6mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/secondary_neutron/simple_294.0kyr_19051.2mwe.npz


 96%|█████████▌| 145/151 [34:47<01:23, 13.91s/it]

    - Saved processed data to Data/processed_recoils/Halite/secondary_neutron/simple_296.0kyr_19180.8mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/secondary_neutron/simple_298.0kyr_19310.4mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/secondary_neutron/simple_300.0kyr_19440.0mwe.npz


100%|██████████| 151/151 [36:01<00:00, 14.31s/it]


In [ ]:
total_tracks_simple_test = [mineral.integrate_all_particles(
    x_bins, 
    scenario_config=scenario_config_simple, 
    energy_bins_gev=energies, 
    exposure_window_kyr=exposure_window_kyr, 
    sample_mass_kg = sample_mass_kg,
    deposition_rate_m_kyr = deposition,
    overburden_density_g_cm3 = continuous_overburden_density_g_cm3,
    steps=300,
) for deposition in deposition_rate_m_kyr]

Processing species: mu-
Integrating the signal in a 0.001 mm slice of Halite with mass 0.1 g, corresponding to 462.96296296296293 cm2


  0%|          | 0/301 [00:00<?, ?it/s]

    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_3.0kyr_16.2mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_5.0kyr_27.0mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_1.0kyr_5.4mwe.npz


  0%|          | 1/301 [00:44<3:40:23, 44.08s/it]

    - Saved processed data to Data/processed_recoils/Halite/mu-/simple_7.0kyr_37.8mwe.npz


...

    - Saved processed data to Data/processed_recoils/Halite/neutron/simple_93.0kyr_502.2mwe.npz


 30%|██▉       | 90/301 [16:58<41:51, 11.90s/it]

    - Saved processed data to Data/processed_recoils/Halite/neutron/simple_95.0kyr_513.0mwe.npz


 31%|███       | 92/301 [17:26<43:56, 12.61s/it]

    - Saved processed data to Data/processed_recoils/Halite/neutron/simple_97.0kyr_523.8mwe.npz
    - Saved processed data to Data/processed_recoils/Halite/neutron/simple_99.0kyr_534.6mwe.npz


In [ ]:
fig, ax = plt.subplots(figsize=[7, 5])

ax.step(x_mids/1000, total_fission_tracks, color='C3', linestyle='dashed', linewidth=1., label="Fission tracks", where='mid')
ax.step(x_mids/1000, total_neutron_tracks, color='C5', linestyle='dashed', linewidth=1., label="Neutron-induced tracks", where='mid')


ax.fill_between(x_mids/1000, total_tracks_simple[0]['total'], total_tracks_simple[-1]['total'], color='C0', alpha=0.2, label='Simple Scenario Range')
ax.fill_between(x_mids/1000, total_tracks_simple_test[0]['total'], total_tracks_simple_test[-1]['total'], color='C0', alpha=0.2, label='Simple Scenario Range')

ax.fill_between(x_mids/1000, 1e-5, 1, color='grey', alpha=0.15)


#ax.set_title(f"Track density distribution in {mineral_name} in the deposition rate range")
ax.set_xlabel(r"Track Length ($\mu$m)")
ax.set_ylabel(r"Number of Tracks")
ax.set_yscale("log")
ax.set_ylim(1e-5, 1e6)
ax.set_xlim(0, 50)
ax.legend(fontsize='small')
plt.show()